# 07 - Counters: Automatic Incrementing

## Introduction

**Counters** are registers that automatically increment (or decrement) their value. The most important counter in a CPU is the **Program Counter (PC)**, which keeps track of the current instruction address.

In this notebook, we'll build:

1. **Binary Counter** - Counts up on each clock cycle
2. **Program Counter** - Counter with load, increment, and reset

## Learning Objectives

1. Understand how counters work using flip-flops and adders
2. Learn about counter control signals
3. Build the Program Counter for our CPU

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))
sys.path.insert(0, str(project_root))

from computer import int_to_bits, bits_to_int
from typing import List
print("Setup complete!")

## Counter Architecture

A counter is essentially a register with an adder in the feedback loop:

```mermaid
flowchart LR
    ONE((1)) --> ADDER[Adder]
    REG[Register] --> |current| ADDER
    ADDER --> |next| REG
    CLK((CLK)) --> REG
    REG --> OUT["Count[7:0]"]
```

On each clock cycle:
1. Current count goes to adder
2. Adder adds 1
3. Result stored back in register

## Exercise 1: 8-bit Binary Counter

In [ ]:
class BinaryCounter8:
    """8-bit binary counter."""
    
    def __init__(self):
        self.count = [0] * 8
    
    def clock(self, enable: int = 1, reset: int = 0, clk: int = 1) -> List[int]:
        """
        Advance counter on clock edge.
        
        Args:
            enable: 1 to count, 0 to hold
            reset: 1 to reset to 0
            clk: Clock signal
        
        Returns:
            Current count value
        """
        # YOUR CODE HERE
        # Priority: reset > enable
        # If reset: count = 0
        # If enable: count = count + 1
        pass
    
    def read(self) -> List[int]:
        """Read current count."""
        return self.count.copy()

# Test
counter = BinaryCounter8()
print("Counting:")
for i in range(10):
    val = counter.clock(1, 0)
    print(f"  {i+1}: {bits_to_int(val)}")

print("\nReset:")
counter.clock(0, 1)  # Reset
print(f"  Count = {bits_to_int(counter.read())}")

## The Program Counter

The **Program Counter (PC)** is a special counter that:

1. **Increments** to the next instruction (normal execution)
2. **Loads** a new value for jumps/branches
3. **Resets** to 0 on startup or reset

```mermaid
flowchart TB
    LoadValue["Load Value"] --> PC[PC]
    Load((Load)) --> PC
    Increment((Increment)) --> PC
    Reset((Reset)) --> PC
    CLK((CLK)) --> PC
    PC --> Address["Address[7:0]"]
```

## Exercise 2: Program Counter

In [ ]:
class ProgramCounter:
    """Program Counter with load, increment, and reset."""
    
    def __init__(self):
        self.value = [0] * 8
    
    def clock(self, load: int, load_value: List[int],
              increment: int, reset: int, clk: int) -> List[int]:
        """
        Update PC based on control signals.
        
        Priority: reset > load > increment
        
        Args:
            load: Load new value (for jumps)
            load_value: Value to load
            increment: Increment by 1 (next instruction)
            reset: Reset to 0
            clk: Clock signal
        
        Returns:
            Current PC value
        """
        # YOUR CODE HERE
        # Priority order: reset > load > increment
        pass
    
    def read(self) -> List[int]:
        """Read current PC value."""
        return self.value.copy()

# Test
pc = ProgramCounter()
zero = [0] * 8

print("Normal execution (increment):")
for i in range(5):
    val = pc.clock(0, zero, 1, 0, 1)
    print(f"  PC = {bits_to_int(val)}")

print("\nJump to address 0x20:")
jump_addr = int_to_bits(0x20, 8)
val = pc.clock(1, jump_addr, 0, 0, 1)
print(f"  PC = {bits_to_int(val)} (0x{bits_to_int(val):02X})")

print("\nContinue from there:")
for i in range(3):
    val = pc.clock(0, zero, 1, 0, 1)
    print(f"  PC = {bits_to_int(val)} (0x{bits_to_int(val):02X})")

print("\nReset:")
val = pc.clock(0, zero, 0, 1, 1)
print(f"  PC = {bits_to_int(val)}")

## Simulating Program Execution

Let's simulate how the PC works during program execution:

In [ ]:
def simulate_execution():
    """Simulate PC behavior during program execution."""
    pc = ProgramCounter()
    zero = [0] * 8
    
    # Simulated instructions
    instructions = [
        ("LOAD R1, 0x10", "increment"),
        ("LOAD R2, 0x11", "increment"),
        ("ADD R0, R1, R2", "increment"),
        ("JMP 0x10", "jump", 0x10),
    ]
    
    print("Program Execution:")
    print("-" * 40)
    
    for instr, action, *args in instructions:
        addr = bits_to_int(pc.read())
        print(f"  PC=0x{addr:02X}: {instr}")
        
        if action == "increment":
            pc.clock(0, zero, 1, 0, 1)
        elif action == "jump":
            jump_addr = int_to_bits(args[0], 8)
            pc.clock(1, jump_addr, 0, 0, 1)
            print(f"         -> Jump to 0x{args[0]:02X}")
    
    print(f"\nFinal PC = 0x{bits_to_int(pc.read()):02X}")

simulate_execution()

## Copy Your Implementation

Once your code works, copy the `BinaryCounter8` and `ProgramCounter` classes to `src/computer/counters.py`

## Validation

In [ ]:
from utils.checker import check
check('counters')

## Summary

We've built the counters our CPU needs:

| Component | Description |
|-----------|-------------|
| BinaryCounter8 | Counts up on each clock |
| ProgramCounter | PC with load/increment/reset |

### Key Concepts

1. **Counters** = Register + Adder in feedback loop
2. **Program Counter** tracks the current instruction
3. **Jumps** load a new value into the PC
4. **Priority**: reset > load > increment

### What's Next?

We now have all the building blocks for storage:
- Flip-flops (single bits)
- Registers (8 bits)
- Counters (auto-increment)

In the next notebook, we'll build **RAM** - the main memory for our computer!